In [1]:
from torchvision.datasets import ImageFolder
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import random_split


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Models.zip

In [5]:
from Data_Loader import *
from Generator import *

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_channels = 3
image_shape = (640,480)
batch_size = 2

In [7]:
device

device(type='cuda')

In [9]:
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])
dataset = ImageFolder('/content/drive/MyDrive/Spectogram/English',transform=data_transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Split the dataset into train and test subsets
train_set, test_set = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
classes = dataset.classes
num_emotions = len(classes)
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features     #extract fc layers features
model.fc = nn.Linear(num_features, num_emotions)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
for name, param in model.named_parameters():
    if param.requires_grad and 'fc' not in name:
        param.requires_grad = False

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 40.3MB/s]


# Training on the real data

In [ ]:
def train_model():
  acc_valid = []
  acc_train = []
  num_epochs = 13   #(set no of epochs)
  start_time = time.time() #(for showing time)
  for epoch in range(num_epochs): #(loop for every epoch)
      print("Epoch {} running".format(epoch)) #(printing message)
      """ Training Phase """
      model.train()    #(training model)
      running_loss = 0.   #(set loss 0)
      running_corrects = 0
      # load a batch data of images
      with tqdm(train_loader,unit="batch") as tepoch:
          tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")
          for inputs, labels in tepoch:
              inputs = inputs.to(device)
              labels = labels.to(device)
              # forward inputs and get output
              optimizer.zero_grad()
              outputs = model(inputs)
              _, preds = torch.max(outputs, 1)
              loss = criterion(outputs, labels)
              # get loss value and update the network weights
              loss.backward()
              optimizer.step()
              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)
              tepoch.set_postfix(loss=loss.item())
      # epoch_loss = running_loss / len(train_data)
      # epoch_acc = running_corrects / len(train_data) * 100.
      # acc_train.append(epoch_acc)
      #print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() -start_time))

      with torch.no_grad():
          running_loss = 0.
          running_corrects = 0
          for inputs, labels in test_loader:
              inputs = inputs.to(device)
              labels = labels.to(device)
              outputs = model(inputs)
              _, preds = torch.max(outputs, 1)
              loss = criterion(outputs, labels)
              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)

          epoch_loss = running_loss / len(test_set)
          epoch_acc = running_corrects / len(test_set) * 100.
          acc_valid.append(epoch_acc)
          print('[Valid #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time()- start_time))

train_model()

c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0 running


  0%|          | 0/63 [00:00<?, ?batch/s]

[Valid #0] Loss: 1.1874 Acc: 53.2000% Time: 103.2698s
Epoch 1 running


  0%|          | 0/63 [00:00<?, ?batch/s]

...

[Valid #10] Loss: 0.3900 Acc: 87.8000% Time: 1051.7460s
Epoch 11 running


  0%|          | 0/63 [00:00<?, ?batch/s]

[Valid #11] Loss: 0.3691 Acc: 88.0000% Time: 1148.8066s
Epoch 12 running


  0%|          | 0/63 [00:00<?, ?batch/s]

[Valid #12] Loss: 0.3624 Acc: 88.0000% Time: 1244.1809s


In [15]:
import os
import shutil
def remove_all_directories(directory):
    for item in os.listdir(directory):
        item_path = os.path.join(directory, item)

        if os.path.isdir(item_path):
            shutil.rmtree(item_path)

In [18]:
def load_model(emotion,path='Models'):
    path_tmp = f'{path}/{emotion}'
    remove_all_directories(path_tmp)

    model_path = os.listdir(path_tmp)[0]
    model_path = f'{path_tmp}/{model_path}'

    gen = Generator(n_channels, n_channels).to(device)
    gen.load_state_dict(torch.load(model_path, map_location=torch.device(device))["gen_AB"])
    return gen


# Evaluating the model on the real and fake data

In [ ]:
from sklearn.metrics import confusion_matrix
predictions = []
true_labels = []
fake_predictions = []
Emotions = ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']
Generators = []
for i,emotion in enumerate(Emotions):
    Generators.append(load_model(emotion))

with torch.no_grad():
  for inputs, labels in test_loader:
      inputs = inputs.to(device)
      labels = labels.to(device)

      tmp = []
      for i in range(inputs.shape[0]):
        fake_image = inputs[i]
        #print(fake_image.shape)
        fake_image = Generators[labels.data[i].item()](fake_image)
        tmp.append(fake_image)
      fake_image = torch.stack(tmp, dim=0)
      fake_outputs = model(fake_image)
      _, fake_predicted = torch.max(fake_outputs, 1)
      fake_predictions.extend(fake_predicted.tolist())

      #fake_image = Generators(get_noise(len(real_image), n_channels,image_shape, device='cuda'))
      outputs = model(inputs)
      _, predicted = torch.max(outputs, 1)
      predictions.extend(predicted.tolist())
      true_labels.extend(labels.tolist())

conf_matrix = confusion_matrix(true_labels, predictions)
conf_matrix

array([[95, 12,  4,  1,  3],
       [ 6, 84,  2,  6,  3],
       [ 1,  0, 84,  5,  1],
       [ 0,  0,  9, 90,  4],
       [ 0,  2,  1,  0, 87]], dtype=int64)

In [19]:
import numpy as np

def calculate_accuracy(conf_matrix):
    diagonal_sum = np.trace(conf_matrix)  # Sum of diagonal elements
    total_sum = np.sum(conf_matrix)       # Total number of samples

    accuracy = diagonal_sum / total_sum   # Calculate accuracy

    return accuracy

In [ ]:
conf_matrix = confusion_matrix(true_labels, fake_predictions)
conf_matrix
print(calculate_accuracy(conf_matrix))

0.744


In [20]:
train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
test_loader = DataLoader(test_set, batch_size=16, shuffle=False)

# Training on the real and fake data

In [22]:
def train_model_sythesized():
  acc_valid = []
  acc_train = []
  num_epochs = 15   #(set no of epochs)
  start_time = time.time() #(for showing time)
  Emotions = ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']
  Generators = []
  for i,emotion in enumerate(Emotions):
    Generators.append(load_model(emotion))

  for epoch in range(num_epochs): #(loop for every epoch)
      print("Epoch {} running".format(epoch)) #(printing message)
      """ Training Phase """
      model.train()    #(training model)
      running_loss = 0.   #(set loss 0)
      running_corrects = 0
      # load a batch data of images
      with tqdm(train_loader,unit="batch") as tepoch:
          tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")
          for inputs, labels in tepoch:
              inputs = inputs.to(device)
              labels = labels.to(device)


              #generating fake images
              tmp = []
              for i in range(inputs.shape[0]):
                    fake_image = inputs[i]
                    #print(fake_image.shape)
                    fake_image = Generators[labels.data[i].item()](fake_image)
                    tmp.append(fake_image)

              tmp = torch.stack(tmp,dim=0)
              inputs = torch.cat((inputs,tmp))
              labels = torch.cat((labels,labels))

              # forward inputs and get output
              optimizer.zero_grad()
              outputs = model(inputs)
              _, preds = torch.max(outputs, 1)
              loss = criterion(outputs, labels)
              # get loss value and update the network weights
              loss.backward()
              optimizer.step()
              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)
              tepoch.set_postfix(loss=loss.item())
      # epoch_loss = running_loss / len(train_data)
      # epoch_acc = running_corrects / len(train_data) * 100.
      # acc_train.append(epoch_acc)
      #print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() -start_time))

      with torch.no_grad():
          running_loss = 0.
          running_corrects = 0
          for inputs, labels in test_loader:
              inputs = inputs.to(device)
              labels = labels.to(device)
              outputs = model(inputs)
              _, preds = torch.max(outputs, 1)
              loss = criterion(outputs, labels)
              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)

          epoch_loss = running_loss / len(test_set)
          epoch_acc = running_corrects / len(test_set) * 100.
          acc_valid.append(epoch_acc)
          print('[Valid #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time()- start_time))

train_model_sythesized()

Epoch 0 running


  0%|          | 0/125 [00:00<?, ?batch/s]

[Valid #0] Loss: 0.7969 Acc: 70.8000% Time: 127.0669s
Epoch 1 running


  0%|          | 0/125 [00:00<?, ?batch/s]

[Valid #1] Loss: 0.8384 Acc: 67.2000% Time: 254.1106s
Epoch 2 running


...

[Valid #12] Loss: 0.6541 Acc: 75.2000% Time: 1676.4650s
Epoch 13 running


  0%|          | 0/125 [00:00<?, ?batch/s]

[Valid #13] Loss: 0.6370 Acc: 75.6000% Time: 1806.1531s
Epoch 14 running


  0%|          | 0/125 [00:00<?, ?batch/s]

[Valid #14] Loss: 0.7512 Acc: 71.6000% Time: 1935.9248s


In [23]:
from sklearn.metrics import confusion_matrix
predictions = []
true_labels = []
fake_predictions = []
Emotions = ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']
Generators = []
for i,emotion in enumerate(Emotions):
    Generators.append(load_model(emotion))

with torch.no_grad():
  for inputs, labels in test_loader:
      inputs = inputs.to(device)
      labels = labels.to(device)

      tmp = []
      for i in range(inputs.shape[0]):
        fake_image = inputs[i]
        #print(fake_image.shape)
        fake_image = Generators[labels.data[i].item()](fake_image)
        tmp.append(fake_image)
      fake_image = torch.stack(tmp, dim=0)
      fake_outputs = model(fake_image)
      _, fake_predicted = torch.max(fake_outputs, 1)
      fake_predictions.extend(fake_predicted.tolist())

      #fake_image = Generators(get_noise(len(real_image), n_channels,image_shape, device='cuda'))
      outputs = model(inputs)
      _, predicted = torch.max(outputs, 1)
      predictions.extend(predicted.tolist())
      true_labels.extend(labels.tolist())

array([[71, 26,  0,  9,  1],
       [ 7, 77,  0,  6,  1],
       [11, 15, 55, 35,  3],
       [ 1,  9,  1, 76,  2],
       [ 3,  8,  1,  3, 79]])

In [26]:
conf_matrix = confusion_matrix(true_labels, predictions)
conf_matrix
calculate_accuracy(conf_matrix)

0.716